Might be two approaches - using threadpool - after order update schedule at time T the change of status of limit order to a solid one
But for back test we will need time speed up and this approach should be used only for engine

For backtest and simulation we will use the following - by getting new snapshot, iterate over all levels and check with the most recent orderbook that is older than 10 seconds, if liquidity was already at this level - it is true liquidity

In [1]:
import pandas as pd
import numpy as np
import random
import time

# Let's simulate the limit order book with 100 price levels
N_LEVELS = 100
# Fixed time T in seconds
T = 10
# Dictionary to store order books with timestamp as key
order_books = {}

# Probabilities for size increase, decrease, or stay the same
PROB_INCREASE = 0.4
PROB_DECREASE = 0.4
# Probability for a level to be empty
PROB_EMPTY = 0.05

def generate_first_order_book():
    rand_price_array = np.random.randint(1, 100, N_LEVELS)
    """Generate a simulated order book with random size and price."""
    order_book = pd.DataFrame({
        'direction': np.random.choice(['buy', 'sell'], N_LEVELS),
        'size': np.random.randint(1, 100, N_LEVELS),
        'price': rand_price_array,
        'true_liquidity': [0]*N_LEVELS,
        'untrue_liquidity': rand_price_array,
    })
    order_book.sort_values('price', ascending=False, inplace=True)
    return order_book

def update_order_book(prev_order_book, curr_time):
    """Update the order book based on the previous order book."""
    order_books[curr_time] = prev_order_book.copy()

    for idx, row in order_books[curr_time].iterrows():
        # Update size based on probabilities
        rand = np.random.uniform()
        if rand < PROB_INCREASE:
            order_books[curr_time].loc[idx, 'size'] += np.random.randint(1, 100)
        elif rand < PROB_INCREASE + PROB_DECREASE:
            order_books[curr_time].loc[idx, 'size'] = max(0, row['size'] - np.random.randint(1, 50))
        # Set size to zero based on probability
        if np.random.uniform() < PROB_EMPTY:
            order_books[curr_time].loc[idx, 'size'] = 0

    # Find the order book strictly older than T seconds
    older_keys = [k for k in order_books.keys() if k < curr_time - T]
    if older_keys:
        older_time = max(older_keys)
        older_order_book = order_books[older_time]

        # Calculate true_liquidity and untrue_liquidity for the current order book
        order_books[curr_time] = order_books[curr_time].merge(
            older_order_book[['direction', 'size', 'price']],
            on=['direction', 'price'],
            how='left',
            suffixes=('', '_older')
        )
        order_books[curr_time]['size_older'].fillna(0, inplace=True)
        order_books[curr_time]['true_liquidity'] = np.minimum(
            order_books[curr_time]['size'],
            order_books[curr_time]['size_older']
        )
        order_books[curr_time]['untrue_liquidity'] = order_books[curr_time]['size'] - order_books[curr_time]['true_liquidity']
        order_books[curr_time].drop(columns='size_older', inplace=True)

    return order_books[curr_time]

# Simulate order book updates
prev_order_book = generate_first_order_book()
start_time = int(time.time())
for i in range(20):
    curr_time = start_time + i
    prev_order_book = update_order_book(prev_order_book, curr_time)


In [2]:
order_books

{1687383625:    direction  size  price  true_liquidity  untrue_liquidity
 5       sell    51     98               0                98
 22       buy     0     98               0                98
 87      sell     0     95               0                95
 29      sell    45     95               0                95
 31       buy     0     94               0                94
 ..       ...   ...    ...             ...               ...
 42      sell    62      7               0                 7
 24      sell    42      6               0                 6
 97       buy    91      6               0                 6
 83       buy    81      3               0                 3
 88       buy     0      2               0                 2
 
 [100 rows x 5 columns],
 1687383626:    direction  size  price  true_liquidity  untrue_liquidity
 5       sell    51     98               0                98
 22       buy    73     98               0                98
 87      sell     0     95        